Goal: Explore Data Provided by WDL on Urban Mobility

In [ ]:
# import necessary packages
import os
import pandas as pd
import numpy as np
from scipy.special import boxcox1p
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# build data directory
data_dir = '../../data/stage1'
os.listdir(data_dir)

### 1: Churn User Profile Data
We will be examining this first

In [ ]:
# Load user profiles
user_profiles_df = pd.read_csv(
    os.path.join(data_dir, 'Churn_UsersProfile.txt'), 
    delimiter = "|",
    encoding = "ISO-8859-1"
)

#### 1.1: Inspect Data

In [ ]:
# sample some user profiles
user_profiles_df.sample(5)

In [ ]:
# No missing data -> Fantastic
user_profiles_df.isna().sum()

In [ ]:
user_profiles_df.info()

#### 1.2: Basic Data Wrangling
-  Make a new column for R1/R2 from Region_of_Origin
-  Make a new column for Region of origin anything after `- AM`
-  Convert columns to categorical
-  Separate period to `period_start`, `period_end` and `period_length`

Assumptions:
-  Period start is on the first day of the month and up to the final day in the last month

In [ ]:
# make new columns from Region_of_Origin
user_profiles_df['origin_region_number'] = user_profiles_df['Region_of_Origin'].str.split(' ').str[0]
user_profiles_df['origin_region'] = user_profiles_df['Region_of_Origin'].str.split('- AM',1).str[1]
user_profiles_df.drop('Region_of_Origin', axis=1, inplace=True)

In [ ]:
# # split period to period_start and period_end (assume from first day of month to last)
# user_profiles_df['period_start'] = user_profiles_df['Period'].str.split('to').str[0]
# user_profiles_df['period_end'] = user_profiles_df['Period'].str.split('to',1).str[-1]
# user_profiles_df.drop('Period', axis=1, inplace=True)

In [ ]:
# from datetime import datetime, timedelta

# user_profiles_df['period_start'] = user_profiles_df['period_start'].apply(lambda x: datetime.strptime(x.strip(), '%b-%y'))
# user_profiles_df['period_end'] = user_profiles_df['period_end'].apply(lambda x: datetime.strptime(x.strip(), '%b-%y'))

# def last_day_of_month(date):
#     if date.month == 12:
#         return date.replace(day=31)
#     return date.replace(month=date.month+1, day=1) - timedelta(days=1)

# user_profiles_df['period_end'] = user_profiles_df['period_end'].apply(last_day_of_month)

In [ ]:
# There are only two periods that we have; seems like wrangling dates wasn't necessary
user_profiles_df['Period'].value_counts()

In [ ]:
# convert types to categorical
to_cat_cols = ['District_of_Origin','County_of_Origin','Period','GenderDescription','AgeClassDescription','origin_region_number','origin_region']
for col in to_cat_cols:
    user_profiles_df[col] = user_profiles_df[col].astype('category')

#### 1.3: Plots

In [ ]:
# Lineplots of Average Bus Users Per Day
fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(8,5))
fig.suptitle('Lineplots of Bus Users Per Day')

ax1.plot(user_profiles_df['Average_BusUsers_per_Day']);
ax1.set(title='Average Bus Users Per Day');

ax2.plot(boxcox1p(user_profiles_df['Average_BusUsers_per_Day'], 0.15));
ax2.set(title='Boxcox Transforation of Bus Users Per Day');
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 5), constrained_layout=True)
fig.suptitle('Histplots of Bus Users Per Day')

sns.histplot(user_profiles_df['Average_BusUsers_per_Day'], ax=axs[0]);
axs[0].set(title='Histplot of Average Bus Users Per Day');

sns.histplot(boxcox1p(user_profiles_df['Average_BusUsers_per_Day'], 0.15), ax=axs[1]);
axs[1].set(title='Histplot of Average Bus Users Per Day Boxcox Tfms');

sns.histplot(np.log1p(user_profiles_df['Average_BusUsers_per_Day']), ax=axs[2]);
axs[2].set(title='Histplot of Average Bus Users Per Day log1p Tfms');
plt.show()

Looks like applying a log1p transformation transforms the distribution to a normal one; we'll do that and plot with it.

In [ ]:
user_profiles_df['log1p_avg_daily_bus_users'] = np.log1p(user_profiles_df['Average_BusUsers_per_Day'])

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='GenderDescription', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
# look at average bus users per day by gender
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users', hue='GenderDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='Period', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='origin_region', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
sns.relplot(
    y='County_of_Origin', 
    x='log1p_avg_daily_bus_users', 
    hue='District_of_Origin', 
    size='log1p_avg_daily_bus_users',
    style='origin_region',
    sizes=(15,200),
    data=user_profiles_df,
    height=8
);


In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users', hue='GenderDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users', hue='Period',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(8,15))
sns.boxplot(x='log1p_avg_daily_bus_users', y='County_of_Origin', hue='District_of_Origin',data=user_profiles_df, orient='h');

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='origin_region_number', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
# pairplot all the categorical data aginst each other
# sns.pairplot(
#     user_profiles_df,
#     vars=to_cat_cols,
#     diag_kind='hist',
#     corner=True,
#     hue='AgeClassDescription'
# );

Findings:
-  Males and females have similar median, upper and lower ranges however there are more extreme cases for female bus users
-  Elderly (65+) have the highest median for usage of busses; middle age (35-44) have the lowest median
-  Female middle age have the lowest bus usage in age range (35-44) and slightly higher than males in age range (15-24)
-  There were slightly more users from Sep-2020 to Jan 2021 than Sep-19 to Feb-20
    -  Slight confusion as Pandemic hit Europe around March 2020? So Sep-20 to Jan-2021 is during the pandemic. Do we discount this information?
    -  Age demographics remain consistent for both of the time periods; elderly rely heavily on public transport
-  Gondomar and Maia have the highest avg_daily_bus_users
-  District of Origin makes a big difference; ages within that vary widely too; Aveiro have the lowest number of daily bus users and Famels from Aveiro even lower
-  Average daily bus users is down from first period to second period. Perhaps this is due to COVID-19, which is the main suspicion at the moment. Though in theory it should be much lower if it were actually COVID? Issue with the data?
-  More people from R2

In [ ]:
user_profiles_df